In [279]:
import nltk
from nltk import word_tokenize
import csv
import numpy as np
np.random.seed(2017)
import pandas as pd
import re
import sys

In [735]:
import itertools
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import keras.backend as K
from sklearn.metrics import accuracy_score, cohen_kappa_score, recall_score, confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
from scipy.stats import pearsonr

In [275]:
data = pd.read_csv('output_data.csv')

In [326]:
data_1 = data[data['essay_set'] == 1]
data_2 = data[data['essay_set'] == 2]
data_3 = data[data['essay_set'] == 3]
data_4 = data[data['essay_set'] == 4]
data_5 = data[data['essay_set'] == 5]
data_6 = data[data['essay_set'] == 6]
data_7 = data[data['essay_set'] == 7]
data_8 = data[data['essay_set'] == 8]

In [818]:
fc_1_results = []

In [819]:
def baseline_model(data_input):
    data_train = data_input[data_input['set'] == 'train']
    data_test = data_input[data_input['set'] == 'test']
    data_dev = data_input[data_input['set'] == 'dev']
    
    normalizer = MinMaxScaler()
    
    X_train = data_train['essay'].tolist()
    y_train = np.asarray(data_train['score'].tolist())
    y_train = normalizer.fit_transform(y_train.reshape(-1, 1))
    
    X_test = data_test['essay'].tolist()
    y_test = np.asarray(data_test['score'].tolist())
    y_test = normalizer.transform(y_test.reshape(-1, 1))

    X_dev = data_dev['essay'].tolist()
    y_dev = np.asarray(data_dev['score'].tolist())
    y_dev = normalizer.transform(y_dev.reshape(-1, 1))
    
    tokenize = text.Tokenizer(num_words=None, char_level=False)
    
    tokenize.fit_on_texts(X_train) # only fit on train
    x_train = tokenize.texts_to_matrix(X_train)
    x_test = tokenize.texts_to_matrix(X_test)
    x_dev = tokenize.texts_to_matrix(X_dev)
    
    # Build the model
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min' , verbose=1, save_best_only=True)
    model = Sequential()
    model.add(Dense(256, input_shape=(len(x_train[0]),), activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])

    history = model.fit(x_train, y_train,
                        batch_size=10,
                        epochs=50,
                        verbose=1, shuffle = False,
                        validation_data=(x_dev,y_dev), callbacks=[es,mc])
    
    return y_test, x_test, y_dev, x_dev, normalizer, history

In [820]:
for input_pair in [[data_1,1],[data_2,2],[data_3,3],[data_4,4],[data_5,5],[data_6,6],[data_7,7],[data_8,8]]:
    for i in range(0,5):
        dataset = input_pair[0]
        y_test, x_test, y_dev, x_dev, normalizer, history = baseline_model(dataset)
        saved_model = load_model('best_model.h5')
        y_d_pred = saved_model.predict(x_dev)
        y_d_T = normalizer.inverse_transform(y_dev).astype(int)
        y_d_P = np.around(normalizer.inverse_transform(y_d_pred)).astype(int)
        y_pred = saved_model.predict(x_test)
        y_T = normalizer.inverse_transform(y_test).astype(int)
        y_P = np.around(normalizer.inverse_transform(y_pred)).astype(int)
        fc_1_results.append([input_pair[1],i,accuracy_score(y_d_T,y_d_P),recall_score(y_d_T,y_d_P,average='weighted'),cohen_kappa_score(y_d_T,y_d_P,weights='quadratic'),accuracy_score(y_T,y_P),recall_score(y_T,y_P,average='weighted'),cohen_kappa_score(y_T,y_P,weights='quadratic'),confusion_matrix(y_T,y_P),history])

Train on 1069 samples, validate on 357 samples
Epoch 1/50
1069/1069 [==============================] - 19s 18ms/step - loss: 0.0297 - mean_absolute_error: 0.1267 - val_loss: 0.0152 - val_mean_absolute_error: 0.0979

Epoch 00001: val_loss improved from inf to 0.01521, saving model to best_model.h5
Epoch 2/50
1069/1069 [==============================] - 10s 10ms/step - loss: 0.0254 - mean_absolute_error: 0.1177 - val_loss: 0.0127 - val_mean_absolute_error: 0.0893

Epoch 00002: val_loss improved from 0.01521 to 0.01265, saving model to best_model.h5
Epoch 3/50
1069/1069 [==============================] - 13s 12ms/step - loss: 0.0106 - mean_absolute_error: 0.0806 - val_loss: 0.0230 - val_mean_absolute_error: 0.1230

Epoch 00003: val_loss did not improve from 0.01265
Epoch 4/50
1069/1069 [==============================] - 13s 12ms/step - loss: 0.0091 - mean_absolute_error: 0.0801 - val_loss: 0.0243 - val_mean_absolute_error: 0.1273

Epoch 00004: val_loss did not improve from 0.01265
Epoch 5

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 1069 samples, validate on 357 samples
Epoch 1/50
1069/1069 [==============================] - 16s 15ms/step - loss: 0.0291 - mean_absolute_error: 0.1240 - val_loss: 0.0159 - val_mean_absolute_error: 0.1013

Epoch 00001: val_loss improved from inf to 0.01594, saving model to best_model.h5
Epoch 2/50
1069/1069 [==============================] - 5s 4ms/step - loss: 0.0207 - mean_absolute_error: 0.1028 - val_loss: 0.0124 - val_mean_absolute_error: 0.0893

Epoch 00002: val_loss improved from 0.01594 to 0.01244, saving model to best_model.h5
Epoch 3/50
1069/1069 [==============================] - 4s 4ms/step - loss: 0.0092 - mean_absolute_error: 0.0730 - val_loss: 0.0199 - val_mean_absolute_error: 0.1128

Epoch 00003: val_loss did not improve from 0.01244
Epoch 4/50
1069/1069 [==============================] - 5s 4ms/step - loss: 0.0094 - mean_absolute_error: 0.0791 - val_loss: 0.0179 - val_mean_absolute_error: 0.1071

Epoch 00004: val_loss did not improve from 0.01244
Epoch 5/50
10


Epoch 00009: val_loss did not improve from 0.01289
Epoch 10/50
1069/1069 [==============================] - 9s 8ms/step - loss: 0.0080 - mean_absolute_error: 0.0727 - val_loss: 0.0139 - val_mean_absolute_error: 0.0941

Epoch 00010: val_loss did not improve from 0.01289
Epoch 11/50
1069/1069 [==============================] - 11s 10ms/step - loss: 0.0079 - mean_absolute_error: 0.0745 - val_loss: 0.0125 - val_mean_absolute_error: 0.0894

Epoch 00011: val_loss improved from 0.01289 to 0.01250, saving model to best_model.h5
Epoch 12/50
1069/1069 [==============================] - 10s 9ms/step - loss: 0.0089 - mean_absolute_error: 0.0813 - val_loss: 0.0166 - val_mean_absolute_error: 0.1040

Epoch 00012: val_loss did not improve from 0.01250
Epoch 13/50
1069/1069 [==============================] - 7s 7ms/step - loss: 0.0091 - mean_absolute_error: 0.0817 - val_loss: 0.0269 - val_mean_absolute_error: 0.1357

Epoch 00013: val_loss did not improve from 0.01250
Epoch 14/50
1069/1069 [===========

Epoch 3/50
1069/1069 [==============================] - 12s 11ms/step - loss: 0.0102 - mean_absolute_error: 0.0794 - val_loss: 0.0281 - val_mean_absolute_error: 0.1383

Epoch 00003: val_loss did not improve from 0.01250
Epoch 4/50
1069/1069 [==============================] - 13s 12ms/step - loss: 0.0076 - mean_absolute_error: 0.0702 - val_loss: 0.0141 - val_mean_absolute_error: 0.0944

Epoch 00004: val_loss did not improve from 0.01250
Epoch 5/50
1069/1069 [==============================] - 14s 13ms/step - loss: 0.0077 - mean_absolute_error: 0.0730 - val_loss: 0.0157 - val_mean_absolute_error: 0.1005

Epoch 00005: val_loss did not improve from 0.01250
Epoch 6/50
1069/1069 [==============================] - 13s 12ms/step - loss: 0.0085 - mean_absolute_error: 0.0770 - val_loss: 0.0143 - val_mean_absolute_error: 0.0947

Epoch 00006: val_loss did not improve from 0.01250
Epoch 7/50
1069/1069 [==============================] - 12s 11ms/step - loss: 0.0076 - mean_absolute_error: 0.0706 - val

...

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Train on 429 samples, validate on 143 samples
Epoch 1/50
429/429 [==============================] - 16s 37ms/step - loss: 0.0517 - mean_absolute_error: 0.1745 - val_loss: 0.0293 - val_mean_absolute_error: 0.1336

Epoch 00001: val_loss improved from inf to 0.02932, saving model to best_model.h5
Epoch 2/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0950 - mean_absolute_error: 0.1951 - val_loss: 0.0251 - val_mean_absolute_error: 0.1267

Epoch 00002: val_loss improved from 0.02932 to 0.02506, saving model to best_model.h5
Epoch 3/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0180 - mean_absolute_error: 0.1004 - val_loss: 0.0296 - val_mean_absolute_error: 0.1355

Epoch 00003: val_loss did not improve from 0.02506
Epoch 4/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0086 - mean_absolute_error: 0.0742 - val_loss: 0.0218 - val_mean_absolute_error: 0.1170

Epoch 00004: val_loss improved from 0.02506 to 0.02179, saving model to b

429/429 [==============================] - 2s 4ms/step - loss: 0.0021 - mean_absolute_error: 0.0378 - val_loss: 0.0312 - val_mean_absolute_error: 0.1393

Epoch 00015: val_loss did not improve from 0.02008
Epoch 16/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0036 - mean_absolute_error: 0.0517 - val_loss: 0.0245 - val_mean_absolute_error: 0.1225

Epoch 00016: val_loss did not improve from 0.02008
Epoch 17/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0045 - mean_absolute_error: 0.0566 - val_loss: 0.0197 - val_mean_absolute_error: 0.1122

Epoch 00017: val_loss improved from 0.02008 to 0.01972, saving model to best_model.h5
Epoch 18/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0067 - mean_absolute_error: 0.0705 - val_loss: 0.0257 - val_mean_absolute_error: 0.1247

Epoch 00018: val_loss did not improve from 0.01972
Epoch 19/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0056 - mean_absolute_error: 0.06

429/429 [==============================] - 2s 4ms/step - loss: 0.0026 - mean_absolute_error: 0.0394 - val_loss: 0.0203 - val_mean_absolute_error: 0.1127

Epoch 00024: val_loss did not improve from 0.01985
Epoch 25/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0016 - mean_absolute_error: 0.0322 - val_loss: 0.0200 - val_mean_absolute_error: 0.1136

Epoch 00025: val_loss did not improve from 0.01985
Epoch 26/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0027 - mean_absolute_error: 0.0418 - val_loss: 0.0203 - val_mean_absolute_error: 0.1148

Epoch 00026: val_loss did not improve from 0.01985
Epoch 27/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0030 - mean_absolute_error: 0.0440 - val_loss: 0.0196 - val_mean_absolute_error: 0.1109

Epoch 00027: val_loss improved from 0.01985 to 0.01964, saving model to best_model.h5
Epoch 28/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0027 - mean_absolute_error: 0.04


Epoch 00023: val_loss did not improve from 0.01963
Epoch 24/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0031 - mean_absolute_error: 0.0487 - val_loss: 0.0216 - val_mean_absolute_error: 0.1207

Epoch 00024: val_loss did not improve from 0.01963
Epoch 25/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0019 - mean_absolute_error: 0.0354 - val_loss: 0.0264 - val_mean_absolute_error: 0.1369

Epoch 00025: val_loss did not improve from 0.01963
Epoch 26/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0021 - mean_absolute_error: 0.0389 - val_loss: 0.0205 - val_mean_absolute_error: 0.1171

Epoch 00026: val_loss did not improve from 0.01963
Epoch 00026: early stopping
Train on 429 samples, validate on 143 samples
Epoch 1/50
429/429 [==============================] - 16s 38ms/step - loss: 0.0517 - mean_absolute_error: 0.1690 - val_loss: 0.0243 - val_mean_absolute_error: 0.1215

Epoch 00001: val_loss improved from inf to 0.02431, savi

In [847]:
fc_1_results_df = pd.DataFrame(fc_1_results)
fc_1_results_df = fc_1_results_df.rename(index=str, columns={0: "essay_set", 1: "run",2:"dev_acc",3:"dev_recall",4:"dev_qwk",5:"test_acc",6:"test_recall",7:"test_qwk",8:"confusion_matrix",9:"history"}) 
fc_1_results_final = fc_1_results_df.groupby("essay_set").mean().reset_index()[['essay_set','dev_acc','dev_recall','dev_qwk','test_acc','test_recall','test_qwk']]

In [904]:
fc_1_results_final

,essay_set,dev_acc,dev_recall,dev_qwk,test_acc,test_recall,test_qwk
0,1,0.636975,0.636975,0.687673,0.645938,0.645938,0.701966
1,2,0.647222,0.647222,0.649414,0.660000,0.660000,0.653048
2,3,0.608696,0.608696,0.575632,0.637572,0.637572,0.589482
3,4,0.670621,0.670621,0.733842,0.649014,0.649014,0.708720
4,5,0.603878,0.603878,0.749617,0.576177,0.576177,0.737741
5,6,0.597222,0.597222,0.767188,0.623889,0.623889,0.771420
6,7,0.262620,0.262620,0.670466,0.239617,0.239617,0.668518
7,8,0.188811,0.188811,0.645400,0.174825,0.174825,0.590709
